<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#6-2-標準的なD&amp;Cアルゴリズム" data-toc-modified-id="6-2-標準的なD&amp;Cアルゴリズム-1">6-2 標準的なD&amp;Cアルゴリズム</a></span><ul class="toc-item"><li><span><a href="#リスト6-1-一般的な分割統治法の実装" data-toc-modified-id="リスト6-1-一般的な分割統治法の実装-1.1">リスト6-1 一般的な分割統治法の実装</a></span></li></ul></li><li><span><a href="#6-3-半分にしながら探索" data-toc-modified-id="6-3-半分にしながら探索-2">6-3 半分にしながら探索</a></span><ul class="toc-item"><li><span><a href="#「ブラックボックス」コラム：bisect" data-toc-modified-id="「ブラックボックス」コラム：bisect-2.1">「ブラックボックス」コラム：<code>bisect</code></a></span></li><li><span><a href="#枝刈りを使った探索木の巡回" data-toc-modified-id="枝刈りを使った探索木の巡回-2.2">枝刈りを使った探索木の巡回</a></span></li><li><span><a href="#リスト6-2-二分探索木の挿入と探索" data-toc-modified-id="リスト6-2-二分探索木の挿入と探索-2.3">リスト6-2 二分探索木の挿入と探索</a></span></li><li><span><a href="#選択アルゴリズム" data-toc-modified-id="選択アルゴリズム-2.4">選択アルゴリズム</a></span></li><li><span><a href="#リスト6-3-分割アルゴリズムと選択アルゴリズムの直感的な実装" data-toc-modified-id="リスト6-3-分割アルゴリズムと選択アルゴリズムの直感的な実装-2.5">リスト6-3 分割アルゴリズムと選択アルゴリズムの直感的な実装</a></span></li></ul></li><li><span><a href="#6−4-半分にしながらソートする" data-toc-modified-id="6−4-半分にしながらソートする-3">6−4 半分にしながらソートする</a></span><ul class="toc-item"><li><span><a href="#リスト6-4-クイックソート" data-toc-modified-id="リスト6-4-クイックソート-3.1">リスト6-4 クイックソート</a></span></li><li><span><a href="#リスト6-5-マージソート" data-toc-modified-id="リスト6-5-マージソート-3.2">リスト6-5 マージソート</a></span></li></ul></li><li><span><a href="#6−5-さらに3つの例" data-toc-modified-id="6−5-さらに3つの例-4">6−5 さらに3つの例</a></span><ul class="toc-item"><li><span><a href="#最大スライス問題" data-toc-modified-id="最大スライス問題-4.1">最大スライス問題</a></span></li></ul></li><li><span><a href="#6−6-木のバランスと...バランスのとり方" data-toc-modified-id="6−6-木のバランスと...バランスのとり方-5">6−6 木のバランスと...バランスのとり方</a></span><ul class="toc-item"><li><span><a href="#リスト6-6-AA木のバランシングを使った二分探索木" data-toc-modified-id="リスト6-6-AA木のバランシングを使った二分探索木-5.1">リスト6-6 AA木のバランシングを使った二分探索木</a></span></li><li><span><a href="#「ブラックボックス」コラム：Binary-Heaps,-Heapq,-Heapsort" data-toc-modified-id="「ブラックボックス」コラム：Binary-Heaps,-Heapq,-Heapsort-5.2">「ブラックボックス」コラム：Binary Heaps, Heapq, Heapsort</a></span></li></ul></li></ul></div>

# 6-2 標準的なD&Cアルゴリズム

## リスト6-1 一般的な分割統治法の実装

以下はこの一般的な分割統治法の流れをコードで表したものです。

In [1]:
def divide_and_conquer(S, divide, combine):
    if len(S) == 1: return S
    L, R = divide(S)
    A = divide_and_conquer(L, divide, combine)
    B = divide_and_conquer(R, divide, combine)
    return combine(A, B)

# 6-3 半分にしながら探索

## 「ブラックボックス」コラム：`bisect`
二分探索は、さまざまな場面で応用できます。「ソートされたシーケンス内のある値を探す」には、標準ライブラリのbisectモジュールがぴったりです。そこには、bisect関数が含まれていて、期待通りに動いてくれます。

In [2]:
from bisect import bisect
a = [0, 2, 3, 5, 6, 8, 8, 9]
bisect(a, 5)
# 4

4

実際、`bisect`は`bisect_right`の別名であり、`bisect_left`もあります。

In [3]:
from bisect import bisect_left
bisect_left(a, 5)
# 3

3

`bisect_right`は以下のようなコードでした。

In [4]:
def bisect_right(a, x, lo=0, hi=None):
    if hi is None:                              # 最後の要素を探す
        hi = len(a)
    while lo < hi:                              # 1つ以上見つかる可能性
        mid = (lo+hi)//2                        # 二分する (真ん中を見つける)
        if x < a[mid]:
            hi = mid                            # Value < middleの場合、左に行く
        else:
            lo = mid+1                          # そうでない場合、右に行く
    return lo

残念なことに、bisectのさまざまな関数は、`list.sort`などで使われている`key`引数をサポートしていま
せん。このような機能は、いわゆるデコレート・ソート・アンデコレート（ここでは、デコレート・探索・
アンデコレート）パターンまたはDSUを使うことで実現できます。

In [5]:
from bisect import bisect_left
seq = "I aim to misbehave".split()
dec = sorted((len(x), x) for x in seq)
keys = [k for (k, v) in dec]
vals = [v for (k, v) in dec]
vals[bisect_left(keys, 3)]
# 'aim'

'aim'

もしくは、以下のようによりコンパクトに書くこともできます。

In [6]:
seq = "I aim to misbehave".split()
dec = sorted((len(x), x) for x in seq)
dec[bisect_left(dec, (3, ""))][1]
# 'aim'

'aim'

## 枝刈りを使った探索木の巡回

## リスト6-2 二分探索木の挿入と探索

以下、辞書のように見せるラッパーと一緒に、二分探索木のシンプルな実装を示しています。

In [7]:
class Node:
    left = None
    right = None

    def __init__(self, key, val):
        self.key = key
        self.val = val


def insert(node, key, val):
    if node is None:
        return Node(key, val)                     # 空の葉の場合: ここにノードを追加
    if node.key == key:
        node.val = val                            # キーを発見した場合: 値を置換
    elif key < node.key:                          # キーより小さい場合：
        node.left = insert(node.left, key, val)   # 左へ行く
    else:                                         # そうでない場合：
        node.right = insert(node.right, key, val) # 右へ行く
    return node


def search(node, key):
    if node is None:
        raise KeyError                          # 空の葉の場合: ここではない
    if node.key == key:
        return node.val                         # キーを発見した場合: 値を返す
    elif key < node.key:                        # キーより小さい場合：
        return search(node.left, key)           # 左へ行く
    else:                                       # そうでない場合：
        return search(node.right, key)          # 右へ行く


class Tree:                                     # シンプルなラッパー
    root = None

    def __setitem__(self, key, val):
        self.root = insert(self.root, key, val)

    def __getitem__(self, key):
        return search(self.root, key)

    def __contains__(self, key):
        try:
            search(self.root, key)
        except KeyError:
            return False
        return True

In [8]:
tree = Tree()
tree["a"] = 42
tree["a"]
# 42

42

## 選択アルゴリズム

## リスト6-3 分割アルゴリズムと選択アルゴリズムの直感的な実装

以下にこの分割アルゴリズムと選択アルゴリズムの実装例を示します。

In [9]:
def partition(seq):
    pi, seq = seq[0], seq[1:]                   # ピボットをランダムに選び、取り除く
    lo = [x for x in seq if x <= pi]            # ピボット以下の要素すべて
    hi = [x for x in seq if x > pi]             # ピボットより大きい要素すべて
    return lo, pi, hi                           # piは「正しい場所」に配置

def select(seq, k):
    lo, pi, hi = partition(seq)                 # [<= pi], pi, [>pi]
    m = len(lo)
    if m == k: return pi                        # k番目の最小値を発見
    elif m < k:                                 # k番目が左側を超えた場合
        return select(hi, k-m-1)                # 左側分を考慮してkを調節
    else:                                       # k番目が右側を超えた場合
        return select(lo, k)                    # 元々のkを選択

実際のシーケンスに対して`partition`と`select`を実行してみましょう。

In [10]:
seq = [3, 4, 1, 6, 3, 7, 9, 13, 93, 0, 100, 1, 2, 2, 3, 3, 2]
partition(seq)
# ([1, 3, 0, 1, 2, 2, 3, 3, 2], 3, [4, 6, 7, 9, 13, 93, 100])

([1, 3, 0, 1, 2, 2, 3, 3, 2], 3, [4, 6, 7, 9, 13, 93, 100])

In [11]:
select([5, 3, 2, 7, 1], 3)
# 5

5

In [12]:
select([5, 3, 2, 7, 1], 4)
# 7

7

In [13]:
ans = [select(seq, k) for k in range(len(seq))]
seq.sort()
ans == seq
# True

True

# 6−4 半分にしながらソートする

## リスト6-4 クイックソート

以下、クイックソートの実装例を示しています。

In [14]:
def quicksort(seq):
    if len(seq) <= 1:
        return seq                              # ベースケース
    lo, pi, hi = partition(seq)                 # piを適切な場所に配置
    return quicksort(lo) + [pi] + quicksort(hi) # loとhiを別々にソートする

In [15]:
seq = [7, 5, 0, 6, 3, 4, 1, 9, 8, 2]
quicksort(seq)
# [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## リスト6-5 マージソート

第3章にてすでにマージソートのコード（リスト3-2）は見ていますが、ここでコメントを加
えながら、もう一度マージソードのコードを見ていきます。

In [16]:
def mergesort(seq):
    mid = len(seq)//2                           # 分割用の真ん中の点
    left, right = seq[:mid], seq[mid:]
    if len(left) > 1:
        left = mergesort(left)                  # 半分をソート
    if len(right) > 1:
        right = mergesort(right)
    res = []
    while left and right:                       # 半分はどちらも空でない限り
        if left[-1] >= right[-1]:               # leftの最後の値が最大値の場合
            res.append(left.pop())              # その最後の値をappendする
        else:                                   # rightの最後の値が最大値の場合
            res.append(right.pop())             # その最後の値をappendする
    res.reverse()                               # 結果を反転する
    return (left or right) + res                # 残りを足して返す

In [17]:
seq = [7, 5, 0, 6, 3, 4, 1, 9, 8, 2]
mergesort(seq)
# [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# 6−5 さらに3つの例

## 最大スライス問題

最大スライス問題は、実数を含んだシーケンスAがあり、その中に`sum(A[i:j])`が最大になるような
スライス（またはセグメント）`A[i:j]`を見つける問題です。
`n=len(A)`とすると、自明な解の例は以下になります。

In [18]:
A = [-1, 2, 1, 0, 4, -3, -6, 1]
n = len(A)
max((A[i:j] for i in range(n) for j in range(i+1, n+1)), key=sum)
# [2, 1, 0, 4]

[2, 1, 0, 4]

これは、単純なブルートフォースを使った解法で、実行時間は三乗時間です！ つまり、非常に非効率です。

以下のように、ある工夫をすることでこの走査を線形にすることができます。

In [19]:
best = A[0]  # A valid solution
for size in range(1, n+1):
    cur = sum(A[:size])
    for i in range(n-size):
        cur += A[i+size] - A[i]
        best = max(best, cur)

best
# 7

7

# 6−6 木のバランスと...バランスのとり方

## リスト6-6 AA木のバランシングを使った二分探索木

AA木に対するすキュー、回転、挿入などの実装を以下に示します。

In [20]:
class Node:
    left = None
    right = None
    level = 1                                     # ノードクラスにレベル属性を追加

    def __init__(self, key, val):
        self.key = key
        self.val = val


def skew(node):                                   # 右回転
    if None in [node, node.left]:
        return node     # スキューがいらない場合その1
    if node.left.level != node.level:
        return node  # スキューがいらない場合その2
    left = node.left                              # 回転のための3ステップ
    node.left = left.right
    left.right = node
    return left                                   # 親からポインタを切り替え


def split(node):                                  # 左回転とレベルを1つ上げる
    if None in [node, node.right, node.right.right]:
        return node
    if node.right.right.level != node.level:
        return node
    right = node.right
    node.right = right.left
    right.left = node
    right.level += 1                              # このノードのレベルを1つ上げる
    return right                                  # このノードがポインタの先に


def insert(node, key, val):
    if node is None:
        return Node(key, val)
    if node.key == key:
        node.val = val
    elif key < node.key:
        node.left = insert(node.left, key, val)
    else:
        node.right = insert(node.right, key, val)
    node = skew(node)                             # 後ろ向きだった場合
    node = split(node)                            # 満杯だった場合
    return node

例として、AA木を文字に変換する関数を以下のように実装します。

In [21]:
def aa_tree_str(root, chunks=None):
    if chunks is None:
        chunks = []
    if root is None:
        chunks.append("*")
    else:
        chunks.append(repr(root.key))
        chunks.append(repr(root.val))
        chunks.append("@")
        chunks.append(repr(root.level))
        chunks.append("{")
        aa_tree_str(root.left, chunks)
        chunks.append(",")
        aa_tree_str(root.right, chunks)
        chunks.append("}")
    return "".join(chunks)

実際にAA木を作り、それを出力してみましょう。

In [22]:
root = None
for key in range(7):
    root = insert(root, key, str(key))

print(aa_tree_str(root))
# 3'3'@3{1'1'@2{0'0'@1{*,*},2'2'@1{*,*}},5'5'@2{4'4'@1{*,*},6'6'@1{*,*}}}

3'3'@3{1'1'@2{0'0'@1{*,*},2'2'@1{*,*}},5'5'@2{4'4'@1{*,*},6'6'@1{*,*}}}


## 「ブラックボックス」コラム：Binary Heaps, Heapq, Heapsort

ヒープを構築する例を以下に示します。

In [23]:
from heapq import heappush, heappop
from random import randrange
Q = []
for i in range(10):
    heappush(Q, randrange(100))

Q
# [15, 20, 56, 21, 62, 87, 67, 74, 50, 74]

[15, 50, 39, 63, 84, 83, 72, 99, 84, 90]

In [24]:
[heappop(Q) for i in range(10)]
# [15, 20, 21, 50, 56, 62, 67, 74, 74, 87]

[15, 39, 50, 63, 72, 83, 84, 84, 90, 99]

`bisect`と同じように、`heapq`モジュールはC言語で実装されていますが、以前はPythonだけで実装されていました。例えば、以下にオブジェクトを両方の子より小さくなるまで下に移動させ続ける関数のコードを示します。

In [25]:
def sift_up(heap, startpos, pos):
    newitem = heap[pos]                         # 動かすことになる要素
    while pos > startpos:                       # 根を超えない限り
        parentpos = (pos - 1) >> 1              # (pos - 1) // 2と同じ意味
        parent = heap[parentpos]                # 父親は誰でしょうか?
        if parent <= newitem:
            break                               # 適合する親が見つかった場合
        heap[pos] = parent                      # そうでない場合: 親を下にコピー
        pos = parentpos                         # 次に調べる要素の位置
    heap[pos] = newitem                         # 要素をその場所に置く